In [4]:
import os
os.chdir("../../../")
os.getcwd()

'C:\\Users\\tsbau\\git\\idp-radio-1'

In [3]:

import keras
import cv2
import pandas as pd
import numpy as np
from skimage.transform import resize
from pathlib import Path
from keras_preprocessing.image import ImageDataGenerator
from keras.applications import resnet_v2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from src.utils.save_model import save_model, model_set

In [5]:
#DATASET_FOLDER = '../../../data/dataset/'
DATASET_FOLDER = 'data/dev_dataset/'
SEED = 17

In [6]:
data = pd.read_csv(os.path.join(DATASET_FOLDER + 'train.csv'))

# preprocess
data = data.fillna(0)

# drop lateral images
data = data[~data['Frontal/Lateral'].str.contains("Lateral")]

# drop unrelevant columns
data = data.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)

# deal with uncertanty (-1) values
data = data.replace(-1,1)
data = data[:64]

np.random.seed(SEED)
data_train, data_test = train_test_split(data, test_size=0.2)
data_train, data_val = train_test_split(data_train, test_size=0.2)

In [7]:
data_train.columns

Index(['Unnamed: 0', 'Path', 'No Finding', 'Enlarged Cardiomediastinum',
       'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
       'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
       'Pleural Other', 'Fracture', 'Support Devices', 'patient_id'],
      dtype='object')

In [8]:
train_datagen=ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [9]:
target_size = (224,224)
train_generator=train_datagen.flow_from_dataframe(
    dataframe=data_train, directory=DATASET_FOLDER , x_col='Path', y_col=list(data_train.columns[2:16]), class_mode='other', target_size=target_size, batch_size=32
)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=data_val, directory=DATASET_FOLDER , x_col='Path', y_col=list(data_val.columns[2:16]), class_mode='other', target_size=target_size, batch_size=32
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data_test, directory=DATASET_FOLDER , x_col="Path", y_col=list(data_test.columns[2:16]), class_mode="other", target_size=target_size, shuffle=False, batch_size=1
)

Found 40 validated image filenames.
Found 11 validated image filenames.
Found 13 validated image filenames.


In [10]:
base_model = resnet_v2.ResNet152V2(include_top=False, weights='imagenet')

# add global pooling and dense output layer 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction_layer = Dense(14, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=prediction_layer)

In [11]:
# freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# compile model
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# fit model 
num_epochs = 3
STEP_SIZE_TRAIN= train_generator.n // train_generator.batch_size
STEP_SIZE_VALID= valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST= test_generator.n // test_generator.batch_size


result = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=num_epochs)

Epoch 1/3
1/1 [==============================] - 15s 15s/step - loss: 0.7018 - accuracy: 0.5156 - val_loss: 0.5238 - val_accuracy: 0.7922
Epoch 2/3
1/1 [==============================] - 4s 4s/step - loss: 0.4765 - accuracy: 0.8304 - val_loss: 0.6452 - val_accuracy: 0.8117
Epoch 3/3
1/1 [==============================] - 4s 4s/step - loss: 0.5649 - accuracy: 0.7857 - val_loss: 0.6491 - val_accuracy: 0.8312


In [14]:
model_id = save_model(model, result.history, 'resnetv2', 'resnet-devdataset.h5')

C:\Users\tsbau\git\idp-radio-1\models\resnetv2\resnet-devdataset.h5


OSError: Unable to create file (unable to open file: name = 'C:\Users\tsbau\git\idp-radio-1\models\resnetv2\resnet-devdataset.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 302)

In [15]:
print("predicting...")
test_generator.reset()
pred=model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

predicting...
13/13 [==============================] - 5s 349ms/step


In [17]:
pred_bool = (pred >= 0.5)
y_pred = np.array(pred_bool, dtype=int)

dtest = data_test.to_numpy()
y_true = np.array(dtest[:,2:16], dtype=int)
report = classification_report(y_true, y_pred, target_names=list(data_test.columns[1:15]))
model_id = model_set(model_id, 'classification_report', report)

C:\Users\tsbau\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsbau\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\tsbau\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\tsbau\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


NameError: name 'model_id' is not defined

In [18]:
score, acc = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)
model_id = model_set(model_id, 'test', (score, acc))

13/13 [==============================] - 2s 186ms/step
Test score: 0.1251792311668396
Test accuracy: 0.8351647257804871


NameError: name 'model_id' is not defined